# Instrument view

## Overview

A simple version of the Mantid [instrument view](https://www.mantidproject.org/MantidPlot:_Instrument_View) is available in `scippneutron`.
It currently does not support detector 'picking' and manual drawing of masks,
nor does it render the actual shape of the detectors (currently it represents them as 2-D squares),
but basic functionalities such as spatial slicing,
as well as navigation through the time-of-flight dimension via a slider, are provided.

<div class="alert alert-info">

**Note**

The file used in this notebook is from the Mantid [training course data](http://sourceforge.net/projects/mantid/files/Sample%20Data/TrainingCourseData.zip/download).

</div>

<div class="alert alert-warning">

**Warning**
    
While you can interact with the 3D view of the instrument,
the buttons and sliders will have no effect in the documentation pages,
as there is no kernel to perform the operations.
These will only work inside a Jupyter notebook.

</div>

We load a file containing a `position` coordinate:

In [ ]:
import numpy as np
import scipp as sc
import scippneutron as scn

In [ ]:
sample = scn.data.powder_sample()['data'].hist()
sample

The `instrument_view` function can now be used to display a 3-D representation:

In [ ]:
scn.instrument_view(sample)

The <img src='https://upload.wikimedia.org/wikipedia/commons/thumb/b/b3/Font_Awesome_5_solid_cube.svg/240px-Font_Awesome_5_solid_cube.svg.png' width="20"/>
button in the toolbar activates a tool that lets you can create spatial cuts to slice your data in 3D.

It is possible to customize the figure using the usual arguments, as well as adjusting the pixel size, e.g.

In [ ]:
scn.instrument_view(
    sample, cmap="magma", vmax=2000.0 * sc.units.counts, norm="log", pixel_size=0.03
)

## Displaying additional components

Additional beamline components can be displayed in the instrument view by providing a `dict` of component settings.
As we can see above, the loaded data array also contains `source_position` and `sample_position` stored as 3-D vector coordinates.
This is in addition to the `position` coordinate, which defines only the positions of detector pixels.

Apart from `position`, the `instrument_view` does not require data arrays to follow a pre-defined naming convention for additional components.
Instead it relies on the aforementioned `dict` of components, which can be customized, e.g., for a particular beamline.

The `dict` key of the component is used as the text label.
Component settings are given in the form of a dictionary defining, e.g., `color`, `size`, and `type`.
We can pick from a limited number of `type`s including `cylinder`, `disk` and `box`:

In [ ]:
sample_settings = {
    'center': sample.meta['sample_position'],
    'color': '#000000',
    'wireframe': True,
    'size': sc.vector(value=[0.3, 0.3, 0.3], unit=sc.units.m),
    'type': 'box',
}
source_settings = {
    'center': sample.meta['source_position'],
    'color': '#FFC133',
    'size': sc.vector(value=[1000, 2000, 1000], unit=sc.units.mm),
    'type': 'cylinder',
}
scn.instrument_view(
    sample, components={'sample': sample_settings, 'source': source_settings}
)

## Defining detector positions

If data comes without pre-defined detector-positions, e.g., when they are not contained in the original file, they can be defined by simply adding a `position` coordinate to a data array:

In [ ]:
nx, ny = (10, 10)
points_x = np.linspace(0, 1, nx)
points_y = np.linspace(0, 1, ny)
xv, yv = np.meshgrid(points_x, points_y)

_x = sc.array(dims=['spectrum'], values=xv.ravel())
_y = sc.array(dims=['spectrum'], values=yv.ravel())
_z = sc.array(dims=['spectrum'], values=np.zeros(100))

data = sc.arange('spectrum', 100.0, unit='counts')
da = sc.DataArray(data)
da.coords['position'] = sc.geometry.position(_x, _y, _z)
da.coords['spectrum'] = sc.arange('spectrum', 100)

scn.instrument_view(da, pixel_size=0.1)